In [ ]:
# import required libraries
import os
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ProbeSettings,
    Environment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

from azureml.core import (
    Experiment,
    Workspace,
    Run
)

from azureml.core import (
    Workspace,
    Environment,
    RunConfiguration,
    ScriptRunConfig,
)
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

from azure.identity import DefaultAzureCredential


In [ ]:
# required definitons
environment_name = 'my-env'
model_name = 'nnunet-devops-test-2'
#Define the size and location of our deployed compute
cpu_cores = 2
memory_gb = 8
location = 'eastus'
service_name = 'my-service'

# required information
experiment_name = 'nnunet-devops-test-2'
entry_scipt = 'src/score.py'
source_directory = '.'
run_id = "d134788d-094b-4cfa-b38b-c2eced07ba53" #"d6c83f1e-97a7-48a1-b4db-a308de7bd9e2" # gets the latest if empty

In [ ]:
workspace = Workspace.from_config()
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

# get experiment
experiment = Experiment(workspace, experiment_name)
# get experiment's latest run id
if run_id:
    latest_run = Run(experiment, run_id) 
else:
    latest_run = next(experiment.get_runs())
    
print(latest_run)
print(type(latest_run))

model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnUNet/"

# model registration procedure
model = Model(path=model_path,
                name=model_name,
                type=AssetTypes.CUSTOM_MODEL)
registered_model = ml_client.models.create_or_update(model)


In [ ]:
#Define configs for the deployment
inference_config = InferenceConfig(
    entry_script=entry_script, environment=env, source_directory=source_directory
)
aci_config = AciWebservice.deploy_configuration(cpu_cores = cpu_cores, memory_gb = memory_gb,location=location)

#Deloy the service
service = Model.deploy(
        workspace=ws,
        name=service_name,
        models=[model],
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True
    )

service.wait_for_deployment(show_output=True)
